In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.metrics import mean_absolute_error as mse
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from numpy import transpose as t

In [36]:
train = pd.read_csv("data/trainsample.csv")
test_original = pd.read_csv("data/testsample.csv")

In [4]:
print(train.shape)
print(test_original.shape)

(423, 53)
(150, 52)


In [5]:
train.columns

Index(['단지코드', '총세대수', '지역', '공가수', '지하철역', '버스정류장', '단지내주차면수', '등록차량수',
       '국민임대', '영구임대', '임대상가', '공공임대(10년)', '장기전세', '행복주택', '공공임대(분납)', '공공분양',
       '공공임대(50년)', '공공임대(5년)', '임대료', '임대보증금', '전용면적6', '전용면적9', '전용면적10',
       '전용면적5', '전용면적8', '전용면적7', '전용면적4', '전용면적13', '전용면적1', '전용면적2', '전용면적3',
       '전용면적12', '전용면적11', '전용면적14', '전용면적15', '자격유형1', '자격유형2', '자격유형3',
       '자격유형4', '자격유형5', '자격유형6', '자격유형7', '자격유형8', '20대(여자)', '30대(여자)',
       '40대(여자)', '50대(여자)', '60대(여자)', '20대(남자)', '30대(남자)', '40대(남자)',
       '50대(남자)', '60대(남자)'],
      dtype='object')

In [6]:
print(train['지역'].nunique())
print(test_original['지역'].nunique())

16
15


In [7]:
train = train.join(pd.get_dummies(train['지역']).iloc[:,:-1])

In [9]:
test = test_original.join(pd.get_dummies(test_original['지역']).iloc[:,:-1])

In [10]:
test['서울']=0

In [11]:
print(test.shape)

(150, 67)


In [12]:
test = test.drop(['단지코드','지역','단지내주차면수'],axis=1)

In [13]:
train['newy'] = train['등록차량수']/train['단지내주차면수']

In [14]:
X = train.drop(['단지코드','지역','newy','등록차량수','단지내주차면수'],axis=1)
y = train['newy']
y = round(y*100)

In [15]:
X = pd.DataFrame(StandardScaler().fit_transform(X),columns=X.columns)

In [16]:
for i in [2,5,10,15,20]:
    clf = RandomForestClassifier(max_depth=i)
    scores = cross_val_score(clf, X, y, cv=5)
    print(i,scores.mean())

/Users/yoorim/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


2 0.025938375350140052


/Users/yoorim/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


5 0.028319327731092435


/Users/yoorim/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


10 0.03305322128851541


/Users/yoorim/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


15 0.028403361344537813


/Users/yoorim/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


20 0.028375350140056023


In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [22]:
clf = RandomForestClassifier(max_depth=15)
clf.fit(X, y)
pred = clf.predict(X_test)

In [23]:
mse(pred,y_test)

0.0

In [24]:
X_train.shape

(296, 64)

In [25]:
test.shape

(150, 64)

## submission

In [26]:
data3 = pd.read_csv("data/sample_submission.csv")

In [27]:
test = pd.DataFrame(StandardScaler().fit_transform(test),columns=test.columns)

In [28]:
pred = clf.predict(test)

In [29]:
pred = pred/100

In [40]:
pred = test_original['단지내주차면수']*pred

In [45]:
data3['num'] = pred

In [46]:
data3.head()

,code,num
0,C1072,676.17
1,C1128,1203.84
2,C1456,547.00
3,C1840,771.06
4,C1332,1100.88


In [47]:
data3.to_csv("data/sample_submission.csv",index=False)